# IOOS SOS SensorML Access and Parsing
7/11,9/2016, [Emilio Mayorga](https://github.com/emiliom/), for [NANOOS](https://github.com/nanoos-pnw).

Access and parse the SensorML responses from an IOOS SOS M1 service endpoint. Extract all/most station-level attributes defined as part of the IOOS convention, and populate a GeoDataFrame from all the station responses. Request may involve getting all stations procedures available in the SOS endpoint, or only a subset of stations based on the station urn's. [OWSLib](https://github.com/geopython/OWSLib/) and [pyoos](https://github.com/ioos/pyoos/) are used for the requests and parsing.

## Import modules and set up low-level utility functions

In [1]:
from datetime import datetime, timedelta
from urllib import urlencode
from collections import OrderedDict
from lxml import etree

import numpy as np
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd

from owslib.sos import SensorObservationService
from owslib.swe.sensor.sml import SensorML, Contact, Documentation
from owslib.util import testXMLValue, testXMLAttribute, nspath_eval
from owslib.namespaces import Namespaces

from pyoos.collectors.ioos.swe_sos import IoosSweSos
from pyoos.parsers.ioos.describe_sensor import IoosDescribeSensor
from pyoos.parsers.ioos.one.describe_sensor import ont

In [2]:
# These functions are all from OWSLib, with minor adaptations

def get_namespaces():
    n = Namespaces()
    namespaces = n.get_namespaces(["sml", "gml", "xlink", "swe"])
    namespaces["ism"] = "urn:us:gov:ic:ism:v2"
    return namespaces

namespaces = get_namespaces()


def nsp(path):
    return nspath_eval(path, namespaces)

## Workhorse function to issue requests and parse the GetCapabilities and SensorML responses

In [3]:
def get_stations_df(sos_url, station_urns_sel=None):
    """ Returns a GeoDataFrame
    """
    # LATER: ADD ERROR TEST/CATCH AFTER EACH WEB REQUEST
    oFrmt = 'text/xml; subtype="sensorML/1.0.1/profiles/ioos_sos/1.0"'

    if station_urns_sel is not None:
        params = {'service': 'SOS', 'request': 'GetCapabilities', 'acceptVersions': '1.0.0'}        
        sosgc = SensorObservationService(sos_url + '?' + urlencode(params))
        station_urns = station_urns_sel
    else:
        sos_collector = IoosSweSos(sos_url)
        station_urns = [urn.name for urn in sos_collector.server.offerings
                        if 'network' not in urn.name.split(':')]
        sos_collector.features = station_urns
        sml_lst = sos_collector.metadata(timeout=200)

    station_recs = []
    for station_idx, station_urn in enumerate(station_urns):
        if station_urns_sel is not None:
            sml_str = sosgc.describe_sensor(procedure=station_urn, outputFormat=oFrmt,
                                            timeout=200)
            sml = SensorML(sml_str)
            
        else:
            sml = sml_lst[station_idx]
            
        #debug MW:
        #first doesn't work:
        #print(sml._root.text_content)
        #print(etree.tostring(sml._root))
        #if station_idx == 0:
        #        print(etree.tostring(sml._root))

        ds = IoosDescribeSensor(sml._root)
        #debug MW:
        #print(str(ds))
        #for var in ds.variables:
        #    print(var)

        pos = testXMLValue(ds.system.location.find(nsp('gml:Point/gml:pos')))

        system_el = sml._root.findall(nsp('sml:member'))[0].find(nsp('sml:System'))

        # Assume there's a single DocumentList/member; will read the first one only.
        # Assume that member corresponds to xlink:arcrole="urn:ogc:def:role:webPage"
        documents = system_el.findall(nsp('sml:documentation/sml:DocumentList/sml:member'))
        if len(documents) > 0:
            document = Documentation(documents[0])
            webpage_url = document.documents[0].url
        else:
            webpage_url = None
        
        contacts = system_el.findall(nsp('sml:contact/sml:ContactList/sml:member'))
        contacts_dct = {}
        for c in contacts:
            contact = Contact(c)
            role = contact.role.split('/')[-1]
            contacts_dct[role] = contact

        sweQuants = system_el.findall(nsp('sml:outputs/sml:OutputList/sml:output/swe:Quantity'))
        quant_lst = [sweQuant.attrib['definition'] for sweQuant in sweQuants]
        parameter_lst = [sweQuant.split('/')[-1] for sweQuant in quant_lst]

        station = OrderedDict()
        station['station_urn'] = station_urn
        station['lon'] = float(pos.split()[1])
        station['lat'] = float(pos.split()[0])

        station['shortName'] = ds.shortName
        station['longName'] = ds.longName
        station['wmoID'] = ds.get_ioos_def('wmoID', 'identifier', ont)

        # Beware that a station can have >1 classifier of the same type
        # This code does not accommodate that possibility
        station['platformType'] = ds.platformType
        station['parentNetwork'] = ds.get_ioos_def('parentNetwork', 'classifier', ont)
        station['sponsor'] = ds.get_ioos_def('sponsor', 'classifier', ont)
        station['webpage_url'] = webpage_url

        station['operatorSector'] = ds.get_ioos_def('operatorSector', 'classifier', ont)
        station['operator_org'] = contacts_dct['operator'].organization
        station['operator_country'] = contacts_dct['operator'].country
        station['operator_url'] = contacts_dct['operator'].url
        # pull out email address(es) too?
        # station_dct['operator_email'] = contacts_dct['operator'].electronicMailAddress

        station['publisher'] = ds.get_ioos_def('publisher', 'classifier', ont)
        station['publisher_org'] = contacts_dct['publisher'].organization
        station['publisher_url'] = contacts_dct['publisher'].url
        # station_dct['publisher_email'] = contacts_dct['publisher'].electronicMailAddress

        station['starting'] = ds.starting
        station['ending'] = ds.ending
        station['starting_isostr'] = datetime.isoformat(ds.starting)
        station['ending_isostr'] = datetime.isoformat(ds.ending)

        station['parameter_uris'] = ','.join(quant_lst)
        station['parameters'] = ','.join(parameter_lst)

        station_recs.append(station)

    stations_df = pd.DataFrame.from_records(station_recs, columns=station.keys())
    stations_df.index = stations_df['station_urn']

    return stations_df

## Select SOS and optionally station urn's, the issue DescribeSensor requests

In [4]:
sos_endpoints = {
    'CeNCOOS': 'http://sos.cencoos.org/sos/sos/kvp',
    'NANOOS': 'http://data.nanoos.org/52nsos/sos/kvp'
}

Select SOS (sos_label) and optionally a list of station urn's. Using `station_urns = None` will result in querying the SOS GetCapabilities to extract all non-network, station offerings. Otherwise, pass a list of `station_urns` to process only those stations.

### The 2nd of these two cells will be run, naturally

In [5]:
sos_label = 'CeNCOOS'

station_urns = [
    'urn:ioos:station:cencoos:Tiburon',
    'urn:ioos:station:cencoos:Carquinez',
    'urn:ioos:station:cencoos:Trinidad',
    'urn:ioos:station:cencoos:Humboldt',
    'urn:ioos:station:cencoos:Monterey',
    'urn:ioos:station:cencoos:SantaCruz',
    'urn:ioos:station:mlml:mlml-sea'
    ]

#station_urns = ['urn:ioos:station:cencoos:Trinidad']
#station_urns = None

In [6]:
sos_label = 'NANOOS'

#station_urns = ['urn:ioos:station:nanoos:apl_chaba']
# station_urns = ['urn:ioos:station:nanoos:nerrs_sosecwq']

station_urns = None

### Request and parse SensorML documents (and possibly SOS GetCapabilities)

In [7]:
stations_df = get_stations_df(sos_endpoints[sos_label], station_urns)

In [8]:
# Assign EPSG:4326 CRS, retrieved from epsg.io
# The OGC WKT crs string is available directly at http://epsg.io/4326.wkt
# or http://spatialreference.org/ref/epsg/4326/ogcwkt/
crs = '''GEOGCS["WGS 84", 
           DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],
             AUTHORITY["EPSG","6326"]],
           PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],
           UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],
         AUTHORITY["EPSG","4326"]]'
'''
geometry = [Point(xy) for xy in zip(stations_df.lon, stations_df.lat)]
stations_gdf = gpd.GeoDataFrame(stations_df, geometry=geometry, crs=crs)

In [9]:
#stations_gdf.head()
#stations_gdf

**From Lance, about what's an active/working station: "I meant by an 'older' station a station that does not have data for the past 7 days."**

In [10]:
station_active_date = datetime.today() - timedelta(days=7)
active_cnt = len(stations_gdf[stations_gdf.ending > station_active_date.isoformat()])
total_cnt = len(stations_gdf)
print("'Active' stations: %d / Total stations: %d" % (active_cnt, total_cnt))

'Active' stations: 33 / Total stations: 65


In [11]:
stations_gdf[stations_gdf.ending > station_active_date.isoformat()]

,station_urn,lon,lat,shortName,longName,wmoID,platformType,parentNetwork,sponsor,webpage_url,...,publisher,publisher_org,publisher_url,starting,ending,starting_isostr,ending_isostr,parameter_uris,parameters,geometry
station_urn,,,,,,,,,,,,,,,,,,,,,
urn:ioos:station:nanoos:apl_nemo,urn:ioos:station:nanoos:apl_nemo,-124.955002,47.973999,APL-UW NEMO Profiler,(APL-UW) UW/NANOOS NEMO Subsurface profiler ne...,None,moored_buoy,NANOOS,None,http://wavechasers.apl.washington.edu/projects...,...,APL-UW,NANOOS,http://nanoos.org,2016-09-10 06:58:52+00:00,2016-10-24 12:28:52+00:00,2016-09-10T06:58:52+00:00,2016-10-24T12:28:52+00:00,http://mmisw.org/ont/cf/parameter/sea_water_pr...,"sea_water_pressure,sea_water_temperature,sea_w...",POINT (-124.9550018310547 47.9739990234375)
urn:ioos:station:nanoos:apl_npb1ptwells,urn:ioos:station:nanoos:apl_npb1ptwells,-122.397202,47.761200,APL-UW NPB-1,(APL-UW) LSG NPB-1 Profiling Buoy at Pt. Wells,46120,moored_buoy,NANOOS,None,http://orca.ocean.washington.edu/data_pointWel...,...,APL-UW,NANOOS,http://nanoos.org,2015-12-04 06:28:48+00:00,2016-10-24 12:55:43+00:00,2015-12-04T06:28:48+00:00,2016-10-24T12:55:43+00:00,http://mmisw.org/ont/cf/parameter/wind_from_di...,"wind_from_direction,sea_water_sigma_t,fraction...",POINT (-122.3972015380859 47.76119995117188)
urn:ioos:station:nanoos:apl_npb2carr,urn:ioos:station:nanoos:apl_npb2carr,-122.730003,47.279999,APL-UW NPB-2,(APL-UW) LSG NPB-2 Profiling Buoy at Carr Inlet,46121,moored_buoy,NANOOS,None,http://orca.ocean.washington.edu/data_carrInle...,...,APL-UW,NANOOS,http://nanoos.org,2015-12-07 20:09:44+00:00,2016-10-24 12:55:44+00:00,2015-12-07T20:09:44+00:00,2016-10-24T12:55:44+00:00,http://mmisw.org/ont/cf/parameter/sea_water_si...,"sea_water_sigma_t,mass_concentration_of_oxygen...",POINT (-122.7300033569336 47.27999877929688)
urn:ioos:station:nanoos:cmop_mottb,urn:ioos:station:nanoos:cmop_mottb,-123.758598,46.203602,CMOP Saturn04,(CMOP) SATURN-04,SEFO3,fixed,NANOOS,None,http://www.stccmop.org/datamart/observation_ne...,...,CMOP,NANOOS,http://nanoos.org,2014-08-02 06:12:56+00:00,2016-10-24 13:04:41+00:00,2014-08-02T06:12:56+00:00,2016-10-24T13:04:41+00:00,http://mmisw.org/ont/cf/parameter/sea_water_tu...,"sea_water_turbidity,cdom,mole_concentration_of...",POINT (-123.7585983276367 46.2036018371582)
urn:ioos:station:nanoos:cmop_saturn03,urn:ioos:station:nanoos:cmop_saturn03,-123.940002,46.199699,CMOP_Saturn03,(CMOP) SATURN-03,SETO3,fixed,NANOOS,None,http://www.stccmop.org/datamart/observation_ne...,...,CMOP,NANOOS,http://nanoos.org,2014-10-23 18:20:26+00:00,2016-10-24 13:05:00+00:00,2014-10-23T18:20:26+00:00,2016-10-24T13:05:00+00:00,http://mmisw.org/ont/cf/parameter/mass_concent...,"mass_concentration_of_oxygen_in_sea_water,H1_F...",POINT (-123.9400024414063 46.19969940185547)
urn:ioos:station:nanoos:cmop_saturn06,urn:ioos:station:nanoos:cmop_saturn06,-122.666702,45.518600,CMOP_Saturn06,(CMOP) SATURN River Station 06 (USGS Station),PRTO3,river_level_station,NANOOS,None,http://www.stccmop.org/datamart/observation_ne...,...,CMOP,NANOOS,http://nanoos.org,2014-08-01 00:30:00+00:00,2016-10-24 00:00:00+00:00,2014-08-01T00:30:00+00:00,2016-10-24T00:00:00+00:00,http://mmisw.org/ont/cf/parameter/water_height...,"water_height_depth,sea_water_ph_reported_on_to...",POINT (-122.6667022705078 45.51860046386719)
urn:ioos:station:nanoos:cmop_saturn07,urn:ioos:station:nanoos:cmop_saturn07,-124.016197,46.286701,CMOP_Saturn07,(CMOP) SATURN-07,None,moored_buoy,NANOOS,None,http://www.stccmop.org/datamart/observation_ne...,...,CMOP,NANOOS,http://nanoos.org,2015-01-22 21:21:03+00:00,2016-10-24 13:05:07+00:00,2015-01-22T21:21:03+00:00,2016-10-24T13:05:07+00:00,http://mmisw.org/ont/cf/parameter/sea_water_sa...,"sea_water_salinity,cdom,mass_concentration_of_...",POINT (-124.0161972045898 46.28670120239258)
urn:ioos:station:nanoos:cmop_saturn08,urn:ioos:station:nanoos:cmop_saturn08,-122.378998,45.577000,CMOP_Saturn08,(CMOP) SATURN-08 (LOBO Station),None,moored_buoy,NANOOS,None,http://w

## Export to GeoPackage (gpkg) file

In [12]:
stations_gdf.dtypes

station_urn                      object
lon                             float64
lat                             float64
shortName                        object
longName                         object
wmoID                            object
platformType                     object
parentNetwork                    object
sponsor                          object
webpage_url                      object
operatorSector                   object
operator_org                     object
operator_country                 object
operator_url                     object
publisher                        object
publisher_org                    object
publisher_url                    object
starting            datetime64[ns, UTC]
ending              datetime64[ns, UTC]
starting_isostr                  object
ending_isostr                    object
parameter_uris                   object
parameters                       object
geometry                         object
dtype: object

In [13]:
for d in stations_gdf:
    print(d, type(stations_gdf['{}'.format(d)][0]))

('station_urn', <type 'str'>)
('lon', <type 'numpy.float64'>)
('lat', <type 'numpy.float64'>)
('shortName', <type 'unicode'>)
('longName', <type 'unicode'>)
('wmoID', <type 'NoneType'>)
('platformType', <type 'str'>)
('parentNetwork', <type 'str'>)
('sponsor', <type 'NoneType'>)
('webpage_url', <type 'str'>)
('operatorSector', <type 'str'>)
('operator_org', <type 'str'>)
('operator_country', <type 'str'>)
('operator_url', <type 'str'>)
('publisher', <type 'str'>)
('publisher_org', <type 'str'>)
('publisher_url', <type 'str'>)
('starting', <class 'pandas.tslib.Timestamp'>)
('ending', <class 'pandas.tslib.Timestamp'>)
('starting_isostr', <type 'str'>)
('ending_isostr', <type 'str'>)
('parameter_uris', <type 'str'>)
('parameters', <type 'str'>)
('geometry', <class 'shapely.geometry.point.Point'>)


In [14]:
# Remove attributes that lead to puking when exporting to file
stations_gdf_out = stations_gdf.drop(['starting', 'ending'], axis=1)

In [15]:
#Display the GDAL drivers available:
import gdal
gdal.AllRegister()

for i in range(1, gdal.GetDriverCount()):
    drv = gdal.GetDriver(i)
    print(drv.GetDescription())

GTiff
NITF
RPFTOC
ECRGTOC
HFA
SAR_CEOS
CEOS
JAXAPALSAR
GFF
ELAS
AIG
AAIGrid
GRASSASCIIGrid
SDTS
DTED
PNG
JPEG
MEM
JDEM
GIF
BIGGIF
ESAT
BSB
XPM
BMP
DIMAP
AirSAR
RS2
SAFE
PCIDSK
PCRaster
ILWIS
SGI
SRTMHGT
Leveller
Terragen
GMT
netCDF
HDF4
HDF4Image
ISIS3
ISIS2
PDS
VICAR
TIL
ERS
L1B
FIT
GRIB
RMF
MSGN
RST
INGR
GSAG
GSBG
GS7BG
COSAR
TSX
COASP
R
MAP
KMLSUPEROVERLAY
CALS
SENTINEL2
MRF
PNM
DOQ1
DOQ2
GenBin
PAux
MFF
MFF2
FujiBAS
GSC
FAST
BT
LAN
CPG
IDA
NDF
EIR
DIPEx
LCP
GTX
LOSLAS
NTv2
CTable2
ACE2
SNODAS
KRO
ROI_PAC
ENVI
EHdr
ISCE
ARG
RIK
USGSDEM
GXF
KEA
BAG
HDF5
HDF5Image
NWT_GRD
NWT_GRC
ADRG
SRP
BLX
SAGA
XYZ
HF2
OZI
CTG
E00GRID
ZMap
NGSGEOID
IRIS
DB2ODBC
ESRI Shapefile
MapInfo File
UK .NTF
OGR_SDTS
S57
DGN
OGR_VRT
REC
Memory
BNA
CSV
NAS
GML
GPX
KML
GeoJSON
OGR_GMT
ODBC
WAsP
PGeo
MSSQLSpatial
OpenFileGDB
XPlane
DXF
Geoconcept
GeoRSS
GPSTrackMaker
PGDUMP
GPSBabel
SUA
OpenAir
OGR_PDS
HTF
AeronavFAA
Geomedia
EDIGEO
SVG
Idrisi
ARCGEN
SEGUKOOA
SEGY
Walk
SXF
Selafin
JML
VDV
TIGER
AVCBin
AVCE00


In [16]:
fpth = '~/ipythonnotebooks/%s_sossml_stations.gpkg' % sos_label
stations_gdf_out.to_file(fpth, driver='GPKG')

DriverError: unsupported driver: 'GPKG'